In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from langfuse import Langfuse
import logfire

In [2]:
load_dotenv(override=True)
os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv("LANGFUSE_SECRET_KEY")
os.environ["LANGFUSE_HOST"] = os.getenv("LANGFUSE_HOST")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
logfire.configure(
    service_name='my_agent_service',
    send_to_logfire=False,
    
)
# This method automatically patches the OpenAI Agents SDK to send logs via OTLP to Langfuse.
logfire.instrument_openai_agents()

<bound method Logfire.info of <logfire._internal.main.Logfire object at 0x7df19f13c050>>

In [36]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("bhavin@advant.xyz")  
    to_email = To("bhavin2004k@gmail.com")
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "bhavin", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)
    return response

res=send_test_email()

202


In [7]:
print(res.headers)

Server: nginx
Date: Wed, 17 Sep 2025 12:56:06 GMT
Content-Length: 0
Connection: close
X-Message-Id: js7lVeUnRzCv5nfl61zmZw
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [12]:
from langfuse import get_client
langfuse = get_client()
print(langfuse.auth_check()) 

True


In [ ]:

async def main():

        result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
        async for event in result.stream_events():
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
loop = asyncio.get_running_loop()
await loop.create_task(main())

TypeError: no_auto_trace() missing 1 required positional argument: 'x'

In [26]:
message = "Write a cold sales email"

results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


16:56:10.697 OpenAI Agents trace: Agent workflow
16:56:10.712   Agent run: 'Professional Sales Agent'
16:56:10.714 OpenAI Agents trace: Agent workflow
16:56:10.715   Agent run: 'Engaging Sales Agent'
16:56:10.717 OpenAI Agents trace: Agent workflow
16:56:10.719   Agent run: 'Busy Sales Agent'
             OpenAI Agents trace: Agent workflow
               Agent run: 'Professional Sales Agent'
16:56:10.722     Responses API with 'gpt-4o-mini'
             OpenAI Agents trace: Agent workflow
               Agent run: 'Engaging Sales Agent'
16:56:10.727     Responses API with 'gpt-4o-mini'
             OpenAI Agents trace: Agent workflow
               Agent run: 'Busy Sales Agent'
16:56:10.731     Responses API with 'gpt-4o-mini'
Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in helping companies like yours streamline their SOC 2 compli

In [20]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [21]:
message = "Write a cold sales email"


async def get_best_email(message: str) -> str:
    with trace("get_best_email"):
        results = await asyncio.gather(
            Runner.run(sales_agent1, message),
            Runner.run(sales_agent2, message),
            Runner.run(sales_agent3, message),
        )
        outputs = [result.final_output for result in results]

        emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

        best = await Runner.run(sales_picker, emails)

        return f"Best sales email:\n{best.final_output}"
res = await get_best_email(message)
print(res)

18:53:56.208 OpenAI Agents trace: get_best_email
18:53:56.209   Agent run: 'Professional Sales Agent'
18:53:56.210   Agent run: 'Engaging Sales Agent'
18:53:56.210   Agent run: 'Busy Sales Agent'
               Agent run: 'Professional Sales Agent'
18:53:56.212     Responses API with 'gpt-4o-mini'
               Agent run: 'Engaging Sales Agent'
18:53:56.215     Responses API with 'gpt-4o-mini'
               Agent run: 'Busy Sales Agent'
18:53:56.217     Responses API with 'gpt-4o-mini'
18:54:06.411   Agent run: 'sales_picker'
18:54:06.412     Responses API with 'gpt-4o-mini'
Best sales email:
Subject: Let’s Make SOC2 Compliance as Fun as Watching Paint Dry! 🎉

Hi [Recipient's Name],

I hope this email finds you well and not buried under a mountain of compliance paperwork!

Let’s cut to the chase: If you’ve ever thought, “Wow, I really love spending my evenings manually preparing for SOC2 audits,” then it’s time to chat. If you’re like the rest of us,

we prefer life without mountains

In [32]:
sender_email = "bhavin@advant.xyz"
receiver_email = 'bhavin2004k@gmail.com'

In [57]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        from_email = Email(sender_email)  # Change to your verified sender
        to_email = To(receiver_email)  # Change to your recipient
        content = Content("text/plain", body)
        mail = Mail(from_email, to_email, "Sales email", content).get()
        sg.client.mail.send.post(request_body=mail)
        # print(response)
        return {"status": "success"}
    except:
        print("Error")
    

send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7df15be18900>, strict_json_schema=True, is_enabled=True)

In [26]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7df1743a7c40>, strict_json_schema=True, is_enabled=True)

In [27]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7df1743a7a60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7df1743a7420>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

In [28]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

19:17:46.715 OpenAI Agents trace: Sales manager
19:17:46.716   Agent run: 'Sales Manager'
19:17:46.717     Responses API with 'gpt-4o-mini'
19:17:54.640     Function: sales_agent1
19:17:54.641     Function: sales_agent2
19:17:54.641     Function: sales_agent3
                 Function: sales_agent1
19:17:54.641       Agent run: 'Professional Sales Agent'
                 Function: sales_agent2
19:17:54.642       Agent run: 'Engaging Sales Agent'
                 Function: sales_agent3
19:17:54.642       Agent run: 'Busy Sales Agent'
                 Function: sales_agent1
                   Agent run: 'Professional Sales Agent'
19:17:54.643         Responses API with 'gpt-4o-mini'
                 Function: sales_agent2
                   Agent run: 'Engaging Sales Agent'
19:17:54.645         Responses API with 'gpt-4o-mini'
                 Function: sales_agent3
                   Agent run: 'Busy Sales Agent'
19:17:54.646         Responses API with 'gpt-4o-mini'
19:18:01.242     Res